# 1. Enviornment setup

In [3]:
import cv2
import numpy as np

# 2. Pre-processing

### 2.1 Filtering out background

Filter out all background and leave a pure pig depth image to process

In [5]:
def img_filter(file_addr):
    
    # Load image
    image = cv2.imread(file_addr)

    # Adjust GMM parameters
    backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=False)
    fgMask = backSub.apply(image)

    # Refine the foreground mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.erode(fgMask, kernel, iterations=2)
    fgMask = cv2.dilate(fgMask, kernel, iterations=2)

    # Convert image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define color range for the pig color
    lower_color = np.array([110, 255, 254])
    upper_color = np.array([255, 255, 255])
    color_mask = cv2.inRange(hsv, lower_color, upper_color)

    # Combine color mask with foreground mask
    fgMask = cv2.bitwise_and(fgMask, color_mask)

    # Find contours from the mask
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the pig and create a mask for it
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        refined_mask = np.zeros_like(fgMask)
        cv2.fillPoly(refined_mask, [largest_contour], 255)
        fgMask = refined_mask

    # Extract the foreground using the refined mask
    foreground = cv2.bitwise_and(image, image, mask=fgMask)
    
    return foreground

Test the filter result

In [12]:
# eg_img = 'Data/WEEK1-2/Week1/20210922/20210922_3330_b/_Depth_4618.png' # Good example
# eg_img = 'Data/WEEK1-2/Week2/20211003/20211003_3342a/_Depth_4489.png' # Good example
eg_img = 'Data/WEEK1-2/Week2/20211003/20211003_3342a/_Depth_6140.png' # Bad example
foreground = img_filter(eg_img)

# image = cv2.imread(eg_img)
# print("Total number of pixels after filtering:", image.shape[0] * image.shape[1])
cv2.imshow('Foreground', foreground)
cv2.waitKey(0)
cv2.destroyAllWindows()

Total number of pixels after filtering: 921600


### 2.2 Count pixels

Count the pixels in the image

In [16]:
def get_pixel_map(img):

    # Convert the foreground to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Flatten the image
    pixels = img.reshape(-1, img.shape[-1])

    # Count unique colors
    unique_colors, counts = np.unique(pixels[pixels.sum(axis=1) != 0], axis=0, return_counts=True)

    # Create a color map
    color_map = {tuple(color): count for color, count in zip(unique_colors, counts)}
    
    return color_map

Test counting pixels

In [17]:
foreground = img_filter(eg_img)
color_map = get_pixel_map(foreground)
print("Total number of pixels in map:", sum(color_map.values()))
print("Color Map:", color_map)

Total number of pixels in map: 45570
Color Map: {(0, 0, 255): 436, (0, 1, 255): 727, (0, 2, 254): 119, (0, 2, 255): 359, (0, 3, 254): 94, (0, 3, 255): 313, (0, 4, 255): 593, (0, 5, 255): 508, (0, 6, 254): 312, (0, 6, 255): 683, (0, 7, 255): 601, (0, 8, 255): 647, (0, 9, 254): 208, (0, 9, 255): 518, (0, 10, 255): 488, (0, 11, 255): 732, (0, 12, 254): 144, (0, 12, 255): 288, (0, 13, 255): 424, (0, 14, 254): 153, (0, 14, 255): 429, (0, 15, 255): 674, (0, 16, 255): 478, (0, 17, 254): 166, (0, 17, 255): 294, (0, 18, 255): 507, (0, 19, 255): 592, (0, 20, 255): 583, (0, 21, 254): 168, (0, 21, 255): 418, (0, 22, 255): 445, (0, 23, 255): 694, (0, 24, 254): 215, (0, 24, 255): 221, (0, 25, 255): 448, (0, 26, 254): 187, (0, 26, 255): 380, (0, 27, 255): 556, (0, 28, 255): 364, (0, 29, 255): 566, (0, 30, 254): 170, (0, 30, 255): 517, (0, 31, 254): 176, (0, 31, 255): 379, (0, 32, 255): 536, (0, 33, 255): 418, (0, 34, 255): 474, (0, 35, 255): 575, (0, 36, 255): 531, (0, 37, 254): 246, (0, 37, 255): 47

# 3. Model training

TODO HERE